In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 02 — Feature Engineering (TPC‑DS SF1000)
# MAGIC 
# MAGIC This notebook transforms Bronze TPC‑DS tables into engineered features for ML.
# MAGIC 
# MAGIC **Goals**
# MAGIC - Join customer, sales, item, and date tables
# MAGIC - Build customer‑level aggregated features
# MAGIC - Write Silver feature tables to Delta
# MAGIC - Prepare a clean ML‑ready dataset for model training
# MAGIC 
# MAGIC **Feature examples**
# MAGIC - Total spend per customer
# MAGIC - Number of transactions
# MAGIC - Average basket size
# MAGIC - Average discount
# MAGIC - Recency features (days since last purchase)
# MAGIC 
# MAGIC These features will feed into the model training notebook.

# COMMAND ----------


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Configuration

# COMMAND ----------

from pyspark.sql import functions as F

catalog = "workspace"
schema = "ml_tpcds"

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

print(f"Using schema: {catalog}.{schema}")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Load Bronze Tables

# COMMAND ----------

store_sales = spark.table(f"{catalog}.{schema}.store_sales_bronze")
customer = spark.table(f"{catalog}.{schema}.customer_bronze")
item = spark.table(f"{catalog}.{schema}.item_bronze")
date_dim = spark.table(f"{catalog}.{schema}.date_dim_bronze")

print("Loaded Bronze tables.")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Build Unified Sales View

# COMMAND ----------

sales_joined = (
    store_sales.alias("s")
    .join(customer.alias("c"), F.col("s.ss_customer_sk") == F.col("c.c_customer_sk"), "inner")
    .join(item.alias("i"), F.col("s.ss_item_sk") == F.col("i.i_item_sk"), "inner")
    .join(date_dim.alias("d"), F.col("s.ss_sold_date_sk") == F.col("d.d_date_sk"), "inner")
    .filter(F.col("c.c_birth_year") > 1980) # limiting due to limited compute
    .select(
        "c.c_customer_sk",
        "c.c_first_name",
        "c.c_last_name",
        "c.c_birth_year",
        "c.c_birth_country",
        "i.i_category",
        "i.i_class",
        "s.ss_quantity",
        "s.ss_sales_price",
        "s.ss_ext_discount_amt",
        "s.ss_ext_list_price",
        "d.d_date",
        "d.d_day_name",
        "d.d_month_seq",
        "d.d_year"
    )
)

sales_joined.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.sales_silver")

print("Unified Silver sales table created.")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Customer-Level Aggregated Features

# COMMAND ----------
# Load the Silver table 
catalog = "workspace"
schema = "ml_tpcds"
import pyspark.sql.functions as F 
sales_silver = spark.table(f"{catalog}.{schema}.sales_silver")


customer_features = (
    sales_silver.groupBy("c_customer_sk")
    .agg(
        F.count("*").alias("num_transactions"),
        F.sum("ss_quantity").alias("total_quantity"),
        F.sum("ss_sales_price").alias("total_spend"),
        F.avg("ss_sales_price").alias("avg_sales_price"),
        F.avg("ss_ext_discount_amt").alias("avg_discount"),
        F.countDistinct("i_category").alias("num_categories_bought"),
        F.max("d_date").alias("last_purchase_date")
    )
)

# Add recency feature
max_date = sales_silver.agg(F.max("d_date")).first()[0]

customer_features = customer_features.withColumn(
    "days_since_last_purchase",
    F.datediff(F.lit(max_date), F.col("last_purchase_date"))
)

customer_features.write.format("delta").mode("overwrite").saveAsTable(
    f"{catalog}.{schema}.customer_features_silver"
)

print("Customer feature table created.")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Build Final ML-Ready Dataset

# COMMAND ----------
catalog = "workspace"
schema = "ml_tpcds"
import pyspark.sql.functions as F 
customer_features = spark.table(f"{catalog}.{schema}.customer_features_silver")

customer = spark.table(f"{catalog}.{schema}.customer_bronze")

ml_ready = (
    customer.alias("c")
    .join(customer_features.alias("f"), F.col("c.c_customer_sk") == F.col("f.c_customer_sk"), "inner")
    .select(
        "c.c_customer_sk",
        "c.c_first_name",
        "c.c_last_name",
        "c.c_birth_year",
        "c.c_birth_country",
        "f.num_transactions",
        "f.total_quantity",
        "f.total_spend",
        "f.avg_sales_price",
        "f.avg_discount",
        "f.num_categories_bought",
        "f.days_since_last_purchase"
    )
)

ml_ready.write.format("delta").mode("overwrite").saveAsTable(
    f"{catalog}.{schema}.customer_features_gold"
)

print("ML-ready Gold feature table created.")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Preview Final Features

# COMMAND ----------

display(ml_ready.limit(20))


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC # Feature Engineering Complete
# MAGIC 
# MAGIC Your Silver and Gold feature tables are now ready for model training.
